## Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
dataset_path = "/content/drive/MyDrive/cashierless-model"

Mounted at /content/drive


## Change to Python 3.6

In [ ]:
%%bash

MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [ ]:
!which conda # should return /usr/local/bin/conda

!conda --version # should return 4.5.4

!which python # still returns /usr/local/bin/python

!python --version # now returns Python 3.6.5 :: Anaconda, Inc.

/usr/local/bin/conda
conda 4.5.4
/usr/local/bin/python
Python 3.6.5 :: Anaconda, Inc.


In [ ]:
%%bash

conda install --channel defaults conda python=3.6 --yes

conda update --channel defaults --all --yes

In [ ]:
!conda --version # now returns 4.8.3

!python --version # now returns Python 3.6.10 :: Anaconda, Inc.

conda 4.10.3
Python 3.6.13 :: Anaconda, Inc.


In [ ]:
import sys
sys.path.append("/usr/local/lib/python3.6/site-packages")

In [ ]:
!conda install tensorflow-gpu==1.15.0 --yes
!conda install -c conda-forge tf_slim --yes

In [ ]:
!pip install pillow
!pip install lxml
!pip install jupyter
!pip install matplotlib
!pip install pandas
!pip install opencv-python
!pip install cython
!pip install scipy
!pip install -q pycocotools
!pip install lvis
!pip install keras==2.1.6

Sourch Example ->  https://muchamadsyaiffudin.medium.com/object-detection-with-custom-dataset-faster-rcnn-on-google-colab-33b373a625eb

Guideline -> https://pub.towardsai.net/training-faster-r-cnn-using-tensorflow-object-detection-api-with-a-custom-dataset-88dd525666fd

In [ ]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!git clone https://github.com/tensorflow/models.git
%set_env PYTHONPATH=/content/models/research:/content/models/research/slim
import os
os.environ['PYTHONPATH'] += ":/content/models"
sys.path.append("/content/models")

In [ ]:
%cd /content/models/research/slim
!python setup.py build
!python setup.py install

In [ ]:
# setup end here

## Using Example Dataset

In [ ]:
# %cd /content
# %mkdir dataset
# %cd /content/dataset


# #!echo "item {\n id: 1\n name: 'dog'\n}" > label_map.pbtxt

# # shared fileid link for googledrive
# fileId = '1xJMfIjUbFBlKoD-LZCaD0HKd62Z2JQrt'

# import os
# from zipfile import ZipFile
# from shutil import copy
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# fileName = fileId + '.zip'
# downloaded = drive.CreateFile({'id': fileId})
# downloaded.GetContentFile(fileName)
# ds = ZipFile(fileName)
# ds.extractall()
# os.remove(fileName)
# print('Extracted zip file ' + fileName)

## Download pre-trained model

model_zoo : https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md

In [ ]:
# # Download pre-trained model

# %mkdir $dataset_path/models
# %cd $dataset_path/models

# import os
# import shutil
# import glob
# import urllib.request
# import tarfile

# MODEL = 'faster_rcnn_inception_v2_coco_2018_01_28'
# MODEL_FILE = MODEL + '.tar.gz'
# DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
# DEST_DIR = 'faster_rcnn_inception_v2'

# # MODEL = 'faster_rcnn_resnet101_coco_2018_01_28'
# # MODEL_FILE = MODEL + '.tar.gz'
# # DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
# # DEST_DIR = 'faster_rcnn_resnet101'

# #if not (os.path.exists(MODEL_FILE)):
# #  opener = urllib.request.URLopener()
# #  opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

# with urllib.request.urlopen(DOWNLOAD_BASE+MODEL_FILE) as response, open(MODEL_FILE, 'wb') as out_file:
#   shutil.copyfileobj(response, out_file)

# tar = tarfile.open(MODEL_FILE)
# tar.extractall()
# tar.close()

# os.remove(MODEL_FILE)
# if (os.path.exists(DEST_DIR)):
#   shutil.rmtree(DEST_DIR)
# os.rename(MODEL, DEST_DIR)

## Augmentation

In [ ]:
%%bash
cd /content
git clone https://github.com/shubhbrth/Augmentation-for-VOC-Pascal.git
pip install imgaug
pip install pascal_voc_writer

Chagne INPUT_DIR and OUTPUT_DIR from augment.py in /content/Augmentation-for-VOC-Pascal as your prefer

INPUT_DIR = '/content/drive/MyDrive/cashierless-model/images/train' <br>
OUTPUT_DIR = '/content/drive/MyDrive/cashierless-model/images/train-augmented'

In [ ]:
%%bash
cd /content/Augmentation-for-VOC-Pascal
rm -rf /content/drive/MyDrive/cashierless-model/images/train-augmented/*
python augment.py

Augmenting input/apple.xml ...


/usr/local/lib/python3.6/site-packages/imgaug/imgaug.py:184: DeprecationWarning: Method `BoundingBoxesOnImage.cut_out_of_image()` is deprecated. Use `BoundingBoxesOnImage.clip_out_of_image()` instead. clip_out_of_image() has the exactly same interface.
  warn_deprecated(msg, stacklevel=3)


## Converting the prepared dataset’s XML files to CSV

In [ ]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
  for directory in ['train-augmented','test']:
    image_path = os.path.join(os.getcwd(), 'images/{}'.format(directory))
    xml_df = xml_to_csv(image_path)
    if directory == "train-augmented":
        xml_df.to_csv('data/train_labels.csv')
    else:
        xml_df.to_csv('data/test_labels.csv')

    print('Successfully converted xml to csv.')

%cd $dataset_path
main()

## Create Tf_Record

In [ ]:
%cd $dataset_path

!python generate_tfrecord.py --csv_input=data/train_labels.csv  --output_path=data/train.record --image_dir=images/train-augmented

!python generate_tfrecord.py --csv_input=data/test_labels.csv  --output_path=data/test.record --image_dir=images/test

/content/drive/MyDrive/cashierless-model


W1004 17:09:01.300842 139795468318592 module_wrapper.py:139] From generate_tfrecord.py:90: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

Successfully created the TFRecords: /content/drive/MyDrive/cashierless-model/data/train.record


W1004 17:09:05.212046 140562067060608 module_wrapper.py:139] From generate_tfrecord.py:90: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W1004 17:09:06.049685 140562067060608 module_wrapper.py:139] From generate_tfrecord.py:49: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/drive/MyDrive/cashierless-model/data/test.record


## Train

In [ ]:
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

/content/models/research


In [ ]:
%cd /content/models/research/object_detection/

model_dir = dataset_path + "/models/training"
pipeline_config_path= dataset_path + "/data/faster_rcnn_resnet101.config"
!python model_main.py \
!  --model_dir=$model_dir \
!  --num_train_steps=20000 \
!  --pipeline_config_path=$pipeline_config_path

## Export

In [ ]:
%cd /content/models/research/object_detection
trained_checkpoint_prefix= dataset_path + "/models/model.ckpt-20000"
output_dir = dataset_path + "/inference_graph"
%rm -r $output_dir
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path=$pipeline_config_path --trained_checkpoint_prefix=$trained_checkpoint_prefix --output_directory=$output_dir

## Demo

In [ ]:
import tensorflow as tf
sys.path.append("..")
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from object_detection.core import post_processing
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from PIL import ImageFont
from IPython.display import display

In [ ]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model  = tf.saved_model.load(output_dir + "/saved_model")
print(detection_model.signatures['serving_default'].inputs)

[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]


In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

  # Apply NMS
  boxes = tf.constant(np.expand_dims(output_dict["detection_boxes"],axis=1))
  scores = tf.constant(np.array(output_dict["detection_multiclass_scores"]))
  nms,_ = post_processing.multiclass_non_max_suppression(
                                          boxes,
                                          scores,
                                          score_thresh=0.5,
                                          iou_thresh=0.5,
                                          max_size_per_class=1)
  boxes,scores,classes = (nms.get(), nms.get_field("scores"), nms.get_field("classes"))

  # Remove N/A boxes at the 1st element
  output_dict['detection_boxes'] = boxes.numpy()[1:]
  output_dict['detection_scores'] = scores.numpy()[1:]
  output_dict['detection_classes'] = (classes).numpy().astype(np.int64)[1:]
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      min_score_thresh=.5,
      line_thickness=8)

  display(Image.fromarray(image_np).resize(size=(600,600)))
  return output_dict

In [ ]:
PATH_TO_TEST_IMAGES_DIR = dataset_path + '/images/validate'
TEST_IMAGE_PATHS = ["1.webp","2.jpg","3.jpg","4.jpg","5.webp","6.jpg","7.jpg","8.jpg","9.jpg","10.jpg","11.jpg","12.jpg","14.jpg"]
# TEST_IMAGE_PATHS = ["☘ (1).jpg"]
PATH_TO_LABELS = dataset_path + '/data/object-detection.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
o_dict = ''
for image_path in TEST_IMAGE_PATHS:
  o_dict = show_inference(detection_model, PATH_TO_TEST_IMAGES_DIR + "/" + image_path)